In [ ]:
# !pip install transformers==4.28.0
# !pip install --upgrade accelerate
# !pip install git+https://github.com/huggingface/accelerate
!pip install transformers==4.28.0

In [ ]:
!pip install pyarabic

In [ ]:
!pip install arabic_reshaper
!pip install python-bidi

--------------------------------------------
> # Arabic Poem Generator




In [ ]:
from google.colab import drive
import numpy as np
from google.colab import files
import shutil
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer \
,TextDataset, DataCollatorForLanguageModeling, pipeline
from sklearn.model_selection import train_test_split
from pyarabic.araby import strip_tashkeel
import tensorflow as tf
import pandas as pd
import re
from wordcloud import WordCloud, STOPWORDS
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib.pyplot as plt

# Reading Data

In [ ]:
#mount google drive to the colab runtime
drive.mount('/content/drive', force_remount = True)

In [ ]:
!unzip -q "/content/drive/MyDrive/Classification Data/Arabic Poem Comprehensive Dataset (APCD).zip" -d "/content/"

In [ ]:
data = pd.read_csv("/content/Arabic Poem Comprehensive Dataset (APCD).csv")
data

# General Look at the data

In [ ]:
print(f"There are a atotal of {data.shape[0]} poem line in the dataset, I'll use the first 50 000 sample  and take البيت column.")

In [ ]:
data = data[['البيت']]
data

In [ ]:
poems_lines = data['البيت'].to_list()[: 50000]
poems_lines[0: 5]

## Data Cleanning:
  1. Remove strips.
  2. Remove arabic punctuation.
  3. Remove multiple spaces.

In [ ]:
# remove special characters
ARABIC_PUNCTUATION = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

def remove_arabic_punctuation(text: str):
    translator = str.maketrans('', '', ARABIC_PUNCTUATION)
    cleaned_text = text.translate(translator)
    return cleaned_text

def remove_multiple_spaces(text):
    cleaned_text = re.sub(' +', ' ', text)
    return cleaned_text

In [ ]:
poems_lines_df = data[['البيت']].head(50000).copy()
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_arabic_punctuation)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(strip_tashkeel)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_multiple_spaces)

# Finetune GPT-2

In [ ]:
poems_lines_df.to_csv(
    r'arabic_full_text.txt',
    header = None,
    index = None,
    sep=' ',
    mode='a'
    )

In [ ]:
file_path = '/content/arabic_full_text.txt'

Splitting the data into train, and test without shuffeling.

In [ ]:
train, test = train_test_split(poems_lines_df.head(50000), test_size=0.15, shuffle=False)

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
test.shape

Write train and test data each one into distinct text file.

In [ ]:
train.to_csv(
    r'train_dataset.txt',
    header=None,
    index=None,
    sep=' ',
    mode='a'
    )

test.to_csv(
    r'test_dataset.txt',
    header=None,
    index=None,
    sep=' ',
    mode='a'
    )

In [ ]:
train.head(3)

Load the tokenizer that is created specialy to the "aragpt-2base model".

In [ ]:
#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

train_path = '/content/train_dataset.txt'
test_path = '/content/test_dataset.txt'

In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
    )

    return train_dataset, test_dataset, data_collator

In [ ]:
def remove_double_qoutes(file_path: str) -> None:

    # Read the file
    with open(file_path, 'r') as file:
        text = file.read()

    # Remove double quotes
    text_without_quotes = text.replace('"', '')

    # Write the modified text back to the file
    with open(file_path, 'w') as file:
        file.write(text_without_quotes)

remove_double_qoutes(train_path)
remove_double_qoutes(test_path)

In [ ]:
train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

In [ ]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")

training_args = TrainingArguments(
    output_dir = "./gpt2-araPoems", #The output directory
    overwrite_output_dir = True, #overwrite the content of the output directory
    num_train_epochs = 15, # number of training epochs
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

## Save the model

In [ ]:
model.save_pretrained('aragpt_final_one')
tokenizer.save_pretrained('aragpt_final_one')

In [ ]:
trainer.save_model('./aragpt_final_one')

# Compress the folder into a zip file
shutil.make_archive("/content/aragpt_final_one", "zip", "/content/aragpt_final_one")


In [ ]:
df.head(150)['similarity'].describe()

In [ ]:
for i in listy:
    if listy.count(i) > 1:
        index_ = listy.index(i)
        second_index = listy[index_ + 1: ].index(i)
        if second_index > k:
            True
        else:
            False